In [5]:
!pip install music21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 55.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.9 MB/s eta 0:00:00


In [6]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [7]:
def get_notes():
    notes = []

    for file in glob.glob("../input/lakh-midi-clean/10cc/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        parsing_notes = None

        try: 
            s2 = instrument.partitionByInstrument(midi)
            parsing_notes = s2.parts[0].recurse() 
        except: 
            parsing_notes = midi.flat.notes

        for unit in parsing_notes:
            if isinstance(unit, note.Note):
                notes.append(str(unit.pitch))
            elif isinstance(unit, chord.Chord):
                notes.append('.'.join(str(n) for n in unit.normalOrder))

    with open('./notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes
        


In [8]:
def prepare_sequences(notes, n_vocab):
 
    sequence_length = 100
    name_of_pitch = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(name_of_pitch))

    inputs_model = []
    output_model = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        inputs_model.append([note_to_int[char] for char in sequence_in])
        output_model.append(note_to_int[sequence_out])

    n_patterns = len(inputs_model)

    # reshape the input into a format compatible with LSTM layers
    inputs_model = numpy.reshape(inputs_model, (n_patterns, sequence_length, 1))
    # normalize input
    inputs_model = inputs_model / float(n_vocab)

    output_model = to_categorical(output_model)

    return (inputs_model, output_model)

In [9]:
def create_network(inputs_model, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(inputs_model.shape[1], inputs_model.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [10]:
def train(model, inputs_model, output_model):
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(inputs_model, output_model, epochs=200, batch_size=128, callbacks=callbacks_list)

In [11]:
def train_network():
    notes = get_notes()
    n_vocab = len(set(notes))
    inputs_model, output_model = prepare_sequences(notes, n_vocab)
    model = create_network(inputs_model, n_vocab)
    train(model, inputs_model, output_model)


In [ ]:
train_network()

Parsing ../input/lakh-midi-clean/10cc/Im_Not_In_Love.3.mid
Parsing ../input/lakh-midi-clean/10cc/Dreadlock_Holiday.3.mid
Parsing ../input/lakh-midi-clean/10cc/Im_Not_In_Love.mid
Parsing ../input/lakh-midi-clean/10cc/Dreadlock_Holiday.1.mid
Parsing ../input/lakh-midi-clean/10cc/Im_Not_In_Love.1.mid
Parsing ../input/lakh-midi-clean/10cc/Dreadlock_Holiday.2.mid
Parsing ../input/lakh-midi-clean/10cc/The_Things_We_Do_for_Love.mid
Parsing ../input/lakh-midi-clean/10cc/Im_Not_In_Love.2.mid
Parsing ../input/lakh-midi-clean/10cc/Dreadlock_Holiday.mid
Parsing ../input/lakh-midi-clean/10cc/Dreadlock_Holiday.4.mid


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 271ms/step - loss: 4.8948
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 278ms/step - loss: 4.1659
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 269ms/step - loss: 4.0405
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 270ms/step - loss: 3.9536
Epoch 5/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 279ms/step - loss: 3.8464
Epoch 6/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 272ms/step - loss: 3.7524
Epoch 7/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 267ms/step - loss: 3.7861
Epoch 8/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 275ms/step - loss: 3.6712
Epoch 9/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 286ms/step - loss: 3.6290
Epoch 10/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 275ms/step - loss: 3.5437
Epoch 11/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 275ms/step - loss: 3.4506
Epoch 12/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 278ms/step - loss: 3.3544
Epoch 13/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 284ms/step - loss: 3.3445
Epoch 14/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 278ms/step - loss: 3.1392
Epoch 15/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
def generate():
    with open('./notes', 'rb') as filepath:
        notes = pickle.load(filepath)

 
    name_of_pitch = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    , normalized_input = prepare_sequences(notes, name_of_pitch, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, inputs_model, name_of_pitch, n_vocab)
    create_midi(prediction_output)

In [ ]:
def prepare_sequences(notes, name_of_pitch, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(name_of_pitch))

    sequence_length = 100
    inputs_model = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        inputs_model.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(inputs_model)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(inputs_model, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (inputs_model, normalized_input)

In [ ]:
def create_network(inputs_model, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(inputs_model.shape[1], inputs_model.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
    model.load_weights('./weights-improvement-199-0.0327-bigger.hdf5')

    return model

In [ ]:
def generate_notes(model, inputs_model, name_of_pitch, n_vocab):

    start = numpy.random.randint(0, len(inputs_model)-1)

    int_to_note = dict((number, note) for number, note in enumerate(name_of_pitch))

    pattern = inputs_model[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
        